In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [2]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
887,6.5,0.300,0.32,2.0,0.044,34.0,90.0,0.99185,3.37,0.68,11.0,7
124,6.4,0.130,0.47,1.6,0.092,40.0,158.0,0.99280,3.21,0.36,9.8,6
577,7.1,0.220,0.33,2.8,0.033,48.0,153.0,0.98990,3.15,0.38,12.7,7
2454,7.1,0.290,0.30,16.0,0.036,58.0,201.0,0.99954,3.30,0.67,9.0,5
2259,7.2,0.390,0.54,1.4,0.157,34.0,132.0,0.99449,3.11,0.53,9.0,6
3920,7.4,0.250,0.37,2.6,0.050,24.0,132.0,0.99138,3.04,0.53,11.2,6
1953,8.1,0.240,0.38,4.3,0.044,49.0,172.0,0.99600,3.37,0.74,10.8,6
3947,6.1,0.300,0.32,2.2,0.042,41.0,142.0,0.98952,3.31,0.44,12.7,7
2321,4.6,0.445,0.00,1.4,0.053,11.0,178.0,0.99426,3.79,0.55,10.2,5
620,6.5,0.460,0.41,16.8,0.084,59.0,222.0,0.99930,3.18,0.58,9.0,5


In [10]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000

In [3]:
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)

In [4]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

In [5]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2024/04/09 10:55:41 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/ramalingam/assignment3.2/mlruns/410441858085864865', creation_time=1712640341781, experiment_id='410441858085864865', last_update_time=1712640341781, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [6]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [7]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160691
  R2: 0.12678721972772689
Save to: /home/ramalingam/assignment3.2/mlruns/410441858085864865/67e6667f11664018b57b984ff1e00f37/artifacts


In [8]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093895
  R2: 0.2022463182289208
Save to: /home/ramalingam/assignment3.2/mlruns/410441858085864865/3b9c5b2095914c8bb15f85ce608a1551/artifacts


In [9]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: /home/ramalingam/assignment3.2/mlruns/410441858085864865/9c7bc75a1c314d82bed05cde1bb08ece/artifacts


In [11]:
# To run Projects and scripts like "MLproject"
# mlflow run . -P alpha=0.42

In [12]:
# Deploy Model locally
# mlflow models serve -m mlruns/0/f5f7c052ddc5469a852aa52c14cabdf1/artifacts/model/ -h 0.0.0.0 -p 1234

In [13]:
# Testing deployed model
# curl -X POST -H "Content-Type: application/json" --data '{"instances":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}' http://0.0.0.0:1234/invocations

In [14]:
"""
You can also simply build a docker image from your model

`mlflow models build-docker -m mlruns/1/d671f37a9c7f478989e67eb4ff4d1dac/artifacts/model/ -n elastic_net_wine`

and run the container with

`docker run -p 8080:8080 elastic_net_wine`.

Or you can directly deploy to AWS sagemaker or Microsoft Azure ML.
"""

'\nYou can also simply build a docker image from your model\n\n`mlflow models build-docker -m mlruns/1/d671f37a9c7f478989e67eb4ff4d1dac/artifacts/model/ -n elastic_net_wine`\n\nand run the container with\n\n`docker run -p 8080:8080 elastic_net_wine`.\n\nOr you can directly deploy to AWS sagemaker or Microsoft Azure ML.\n'

In [17]:
# Tagging
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.search_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

[<Experiment: artifact_location='/home/ramalingam/assignment3.2/mlruns/410441858085864865', creation_time=1712640341781, experiment_id='410441858085864865', last_update_time=1712640341781, lifecycle_stage='active', name='ElasticNet_wine', tags={}>, <Experiment: artifact_location='/home/ramalingam/assignment3.2/mlruns/0', creation_time=1712640166273, experiment_id='0', last_update_time=1712640166273, lifecycle_stage='active', name='Default', tags={}>]


In [18]:
# get the run
_run = client.get_run(run_id="3b9c5b2095914c8bb15f85ce608a1551")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6155290394093895,
 'r2': 0.2022463182289208,
 'rmse': 0.7859129997062342}, params={'alpha': '0.2', 'l1_ratio': '0.2'}, tags={'mlflow.log-model.history': '[{"run_id": "3b9c5b2095914c8bb15f85ce608a1551", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-04-09 05:41:33.444014", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.12.2", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.3.0", '
                             '"serialization_format": "cloudpickle", "code": '
                             'n

In [19]:
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)